# 7장 순수함수적 병렬성

목표 
- 병렬 및 비동기 계산의 생성을 위한 순수 함수적 라이브러리 하나를 구축
- 순수 함수적 라이브러리의 설계 문제에 대한 접근방식을 배움.
        

## 7.1 자료형식과 함수의 선택

- 분할정복 알고리즘을 이용한 목록 합산
- 이 구현은 병렬화가 가능

In [2]:
def sum( ints : IndexedSeq[Int] ) : Int = {
    
    if ( ints.size <= 1 ) 
        ints.headOption  getOrElse 0 
    else {
        val (l, r) = ints.splitAt( ints.length / 2 )
        sum( l ) + sum( r )
    }
}

defined function sum

In [3]:
val seqdata = IndexedSeq( 1 , 2  , 3 , 4, 5 ,6 ,7, 8, 9, 10)
sum( seqdata )

seqdata: IndexedSeq[Int] = Vector(
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10
)
res1_1: Int = 55

###  이번장에서 구현할려고 하는 쓰레드가 어떤 형태인지 알아보기 위한 문제.. <br/>
A상점에서 치킨을 사는데 걸리는 시간 : 10분 <br/>
B상점에서 족발을 사는데 걸리는 시간 : 10분 <br/>
C상점에서 피자를 사는데 걸리는 시간 : 10분 <br/>
A, B, C 가게 모두에서 물건을 하나씩 살때 가장 효율적인 방법은 ??<br/><br/><br/><br/><br/><br/><br/><br/><br/>


### 자바의 Future 패턴

![FuturePattern](future-1.jpg)

### 7.1.1 병렬계산을 위한 자료형식


- 표현식 sum( l )  + sum( r ) 을 보면, 절반에 대한 재귀적으로 sum()을 호출하므로 
- 병렬계산을 나타내는 자료형식이 하나의 결과를 담을 수 있어야 함. 
- 결과를 담을 컨테이너 형식 Par[ A ] 을 정의하자.

- def unit[A] ( a : => A ) : Par[A]  -   평가되지 않은 A을 받고, 그것을 개별적인 스레드에서 평가할 수 있는 계산을 돌려줌.
- def get[A] ( a: Par[A])  : A -- 병렬계산에서 결과값을 추출


In [ ]:
object Par {
   def unit[A] ( a : => A ) : Par[A]

   def get[A] ( a: Par[A])  : A
}

In [ ]:
def sum( ints : IndexedSeq[Int] ) : Int = {
    
    if ( ints.size <= 1 ) 
        ints.headOption  getOrElse 0 
    else {
        val (l, r) = ints.splitAt( ints.length / 2 )
        val sumL = Par[Int] = Par.unit( sum(l) )  # 왼쪽 절반을 병렬로 계산
        val sumR = Par[Int] = Par.unit( sum(r) )  # 오른쪽 절반을 병렬로 계산
        Par.get(sumL) + Par.get(sumR)             # 두 결과를 추출해서 합산  
    }
}

### 문제가 발생함. 
- 아래와 같이 치완되기 때문에 프로그램이 병렬로 실행되지 않음.
- 왼쪽 절반이 계산이 끝나야지만 오른쪽 절반이 계산이 시작됨.

In [ ]:
Par.get(  Par.unit( sum(l) )  ) + Par.get( Par.unit( sum(r) ) ) 

### 7.1.2 병렬계산의 조합

- unit과 get 조합의 문제점을 피하기 위해서 get을 호출하지 않고 sum함수 구현을 해보면...

In [ ]:
def sum( ints: IndexedSeq[Int] ) : Par[Int] = 
  if( ints.size <= 1 )
    Par.unit( ints.headOption getOrElse 0 )
  else {
      val (l, r) = ints.splitAt( ints.length / 2 )
      Par.map2( sum(l), sum(r) ) ( _ + _ )
  }

### 연습문제 7.1
Par.map2 함수의 서명은 무엇인가?  최대한 일반적인 서명으로 ...

In [ ]:
# 첫번재 풀이
def Par.map2[A] ( l : Par[A], r : Par[A] ) ( f : A => A ) = Par[A] {
    f(l, r)
}




In [ ]:
# 정답
object Par {
    // ....
   def map2[A,B,C](a: Par[A], b: Par[B])(f: (A,B) => C): Par[C]
}

- sum( IndexedSeq( 1 , 2  , 3 , 4 ) )의 평가가 어떻게 진행되는지 확인해보자. 
- 그리고 병렬처리 설계를 어떻게 할지 고민해보자.

In [ ]:
sum( IndexedSeq( 1 , 2  , 3 , 4 ) )



map2 ( 
  sum( IndexedSeq( 1 , 2 ) ),
  sum( IndexedSeq( 3 , 4 ) )
  ) ( _ + _ )



map2 ( 
  map2 ( 
      sum( IndexedSeq( 1 ) ),
      sum( IndexedSeq( 2 ) )
      ) ( _ + _ ), 
  sum( IndexedSeq( 3 , 4 ) )
  ) ( _ + _ )



map2 ( 
  map2 ( 
      unit( 1 ) ,
      unit( 2 )
      ) ( _ + _ ), 
  sum( IndexedSeq( 3 , 4 ) )
  ) ( _ + _ )



map2 ( 
  map2 ( 
      unit( 1 ) ,
      unit( 2 )
      ) ( _ + _ ), 
  map2 ( 
      IndexedSeq( 3 ),
      IndexedSeq( 4 )
     ) ( _ + _ )
  ) ( _ + _ )



map2 ( 
  map2 ( 
      unit( 1 ) ,
      unit( 2 )
      ) ( _ + _ ), 
  map2 ( 
      unit( 3 ),
      unit( 4 )
     ) ( _ + _ )
  ) ( _ + _ )



 ### 병렬처리를 위한 3가지 고민
- 1) map2을 엄격한 함수로 만들면, 병렬처리가 되지 않거나,  스레드풀 같은 것을 사용하면 , 오른쪽 절반이 구축되기전에 왼쪽 절반이 실행됨.
- 2) map2를 엄격하게 유지하되 즉시 실행되지 않게 하면, 수행할 연산들의 전체 트리를 담아 놓아야해서 많은 공간을 차지하게 됨.
- 3) map2를 게으르게 만들면, 양변에 동등한 실행 기회를 부여할 수 있음.

### 7.1.3 명시적 분기

In [ ]:
Par.map2 ( unit( 1 ) , unit( 2 ) ) ( _ + _ )

- map2의 두 인수를 병렬로 평가가 항상 바람직한가 ???
- 위의 코드는 논리적 스레드를 띄울 필요가 없으며, 명시적으로 지정되지 않음.
- 분기를 좀더 명시적으로 표현해보자.

In [ ]:
object Par {
    // ....
    def fork[A] ( a : => Par[A]) : Park[A]
}

def sum( ints: IndexedSeq[Int] ) : Par[Int] = 
  if( ints.size <= 1 )
    Par.unit( ints.headOption getOrElse 0 )
  else {
      val (l, r) = ints.splitAt( ints.length / 2 )
      Par.map2( Parm.fork(sum(l)), Parm.fork(sum(r)) ) ( _ + _ )
  }

- fork 덕분에 map2를 엄격한 함수로 만들고, 인수들을 감싸는 것은 프로그래머의 뜻에 맡길 수 있음.
- 관심 1) 두병렬 과제의 결과들이 결합(조합)되어야 함을 지정하는 수단 => ( _ + _ )
- 관심 2) 특정 과제를 비동기적으로 수행할지 아닐지를 선택하는 수단 => fork
- 관심1) 과 관심2) 를 분리한 덕분에 map2나 기타 조합기에서 병렬성에 관한 어떤 방침(코드)도 내장할 필요가 없음.

### unit을 엄격하게 할지 비엄격하게 할지 결정이 필요
- fork가 있어서 unit은 엄격하게 표현할 필요가 없고, 비엄격 버전을 따라 구현

In [ ]:
def unit[A] ( a : A ) : Par[A]
def lazyUnit[A] ( a : => A) : Par[A] = fork( unit(a) )

### 개별 논리적 스레드에서 평가를 미룰 것인지를 결정하는것을 fork가 책임인지, get의 책임인지의 문제를 결정해보자.
- 문제 해결 요령 : 여러 의미를 가진 fork와 get의 구현에 <font color='red'><b>어떤 정보가 필요한가</b></font>를 생각해보자.

fork가 스레드에서 평가를 미룰것인지를 결정하면, 스레드 생성하는 방법과 과제를 스레드풀에 제출하는 방법을 알아야 하기 때문에 <br/>
fork는 스레드풀 내부를 접근 가능해야 하고 fork를 호출하는 시점에서 스레드풀이 초기화되어야 됨. <br/>
=> 스레드풀 객체가 내부 은닉 특성이 없어짐.<br/>

그러므로, 스레드 생성과 실행과제 제출의 책임을 get에 부여하는것이 좋음.<br/>
get함수 이름을 run()으로 변경하자.

In [ ]:
def run[A] (a : Par[A]) A 

## 7.2 표현의 선택

Par를 위한 API 개요

In [ ]:
object Par {
  def unit[A] ( a : A ) : Par[A] 
  def map2[A,B,C](a: Par[A], b: Par[B])(f: (A,B) => C): Par[C]
  def fork[A] ( a : =>  Par[A] ) : Par[A]
  def lazyUnit[A] ( a : => A) : Par[A] = fork( unit(a) )
  def run[A] (a : Par[A]) : A 
}

- unit은 상수 값을 병렬계산으로 승격한다.
- map2는 두 병렬계산의 결과들을 이항함수로 조합한다.
- fork는 주어진 인수가 동시적으로 평가될 계산임을 표시한다. 그 평가는 run에 강제되어야 실제로 실행된다.
- lazyUnit은 평가되지 않은 인수를 Par로 감싸고, 그것을 병렬 평가 대상으로 표시한다.
- run은 계산을 실제로 실행해서 Par로부터 값을 추출한다.

### 쓰레드 처리를 위한 클래스 정의 
### Java 표준 라이브러리의 java.util.concurrent.ExecutorService를 스칼라로 옮김

In [ ]:
class ExecutorService {
    def submit[A] ( a : Callable[A] ) : Future[A]
} 
trait Callable[A] { def call : A }   // 사실상 그냥 게으른 A 임
trait Future[A] {
    def get : A
    def get( timeout : Long, unit : TimeUnit ) : A 
    def cancel( eventIfRunning: Boolean ) : Boolean 
    def isDone : Boolean 
    def isCancelled : Boolean 
}

### 위의 쓰레드 클래스를 이용해서  Par와 run을 정의하자

In [ ]:
type Par[A] = ExecutorService => Future[A]
def run[A]( s : ExecutorService) ( a : Par[A] )  : Future[A] = a( s )

## 7.3. API 정리

지금까지 개발한 API의 함수들을 구현<br/>
Par의 기본적인 구현

In [ ]:
object Par { 
  def unit[A] ( a : A ) : Par[A] = (es : ExecutorService)  =>  UnitFuture(a)
    
  private case class UnitFuture[A] ( get : A ) extends Future[A] {
      def isDone = true
      def get( timeout : Long, units : TimeUnit) = get 
      def isCancelled = false 
      def cancel(eventIfRunning : Boolean) : Boolean = false
  }

  def map2[A,B,C](a: Par[A], b: Par[B])(f: (A,B) => C): Par[C] = 
    (es : ExecutorService) => {
        val af = a( es )
        val bf = b( es )
        UnitFuture (  f(af.get, bf.get)  )
  }
    
  def fork[A] ( a : =>  Par[A] ) : Par[A] = 
    es => es.submit( new Callable[A] {
        def call = a(es).get
    })
  }

// 아래 문제를 위해서 추가됨.
  def equals[A] ( e : ExecutorService ) ( p : PAR[A], p2 : PAR[A] ) : Boolean = 
    p(e).get == p2(e).get 
    
}

- fork() 함수만이 병렬성 제어를 담당
- map2의 이 구현은 만료( timeout )시간을 설정하지 않음. 
- Future의 af와 bf의 결과들을 기다리고, f를 적용함. => 어떤 문제가 있을지 고민이 필요

### 7.4.3 미묘한 버그 하나

- 내부적으로 고정된 크기의 스레드풀을 사용하는 ExecutorService 구현은 교착( dead locak )에 빠지가가 쉽음.
- 예) ExecutorService가 사용하는 스레드 풀의 최대 스레드 수가 1이라고 할때, 현재 구현을 이요해서 아래 코드를 실행하면 교착상태가 됨.

In [ ]:
val a = lazyUnit( 42 + 1 )
val s = Executors.newFixedThreadPool( 1 )
println(  Par.equals(s)(a, fork(a) )   )

- 그 이유는
- 1) ExecutorService의 스레드풀의 크기가 1이면, 바깥쪽 callable이 제출
- 2) 스레드풀의 유일한 스레드를 할당하고, 그 스레드가 다른 내부의 callable을 제출하고 기다림.
- 3) 내부의 callable은 스레드풀에서 스레드가 할당되기를 기다리지만, 먼저 할당된 스레드가 실행이 끝나지 않았기 때문에 교착상태에 빠짐.

In [ ]:

  def fork[A] ( a : =>  Par[A] ) : Par[A] = 
    es => es.submit( new Callable[A] {
        def call = a(es).get  // => Callable안에서 또 다른 Callable을 ExecutorService에 제출
    })
  }


- 여기에서이 본질적은 문제는 Future의 get 메소드를 호출하지 않으면, Future에서 값을 꺼낼수 없는데,  이 get 메소드가 차단( blocking )된다는 점
- 스레드와 같은 자원을 독점적으로 점유하지 않게 하기 위해서는 비차단( non-blocking )방식으로 변경해야 함.

### 7.4.4 행위자를 이용한 완전 비차단 Par구현

### Par의 비차단 방식으로 변경 
- Par를 값을 꺼내는 방식(java.util.concurrent.Future )을 대신해서 적당한 때에 호출되는 콜백을 등록할 수 있는 방식의 새로운 Future을 구현

In [ ]:
sealed trait Future[A] {
    private [parallelism] def apply( k : A => Unit ) : Unit 
}
type Par[+A] = ExecutorService => Future[A]

- 새로운 Par형식을 이전과 동일하지만, 새로운 Future을 사용
- 새로운 Future는 A형식을 인자로 받고, Unit을 리턴하는 함수 K 받고, 이를 수행하는 apply 메서드를 제공
- 이런 종류를 콜백함수라고 함.

### 비차단 Par를 위한 run의 구현

In [ ]:
def run[A] ( es : ExecutorService ) ( p : Par[A] ) : A = {
    val ref = new AtomicReference[A]
    val latch = new CountDownLatch( 1 )
    p( es ) { a => ref.set(a);  latch.countDown }
    latch.await
    ref.get
}

In [ ]:
def unit[A] ( a : A ) : Par[A] = 
  es => new Future[A] {
      def apply( cb  : A  => Unit ) : Unit = 
        cb( a )
  }

In [ ]:
def fork[A]( a:  =>  Par[A] ) : Par[A] = 
    ese => new Future[A] {
        def apply( cb : A => Unit ) : Unit = 
            eval( es ) ( a(es)(cb) )
    }

In [ ]:
def eval( es : ExecutorService ) ( r : => Unit ) : Unit =
    es.submit( new Callable[Unit]  { def call = r  } )

### scala의 행위자 소개 - actor

- actor는 비차단( non-block ) 동시성 기본 수단을 사용
- 하나의 동시적 프로세스로 스레드를 계속 차지하지 않음.
- actor는 메시지를 받았을때만 스레드를 점유 ( 사용 )
- actor를 사용해서 여러 스레드가 접근해야 하는 까다로운 코드를 작성할때 유용


In [ ]:
import fpinscala.parallelism._
import java.util.concurrent.{Executors, ExecutorService}

val S = Executors.newFixedThreadPool( 4 )
val echoer =  Actor[ String ] ( S ) {
    msg => println( s"Got message :  '$msg' " )
}

echoer ! "hello"

### Actor를 이용한 map2 구현

In [ ]:
def map2[ A, B, C ] ( p : Par[A], p2 : Par[B] ) ( f: (A,B) => C ) : Par[C] = 
  es => new Future[C] {
      def apply( cb : C => Unit ) : Unit = {
          var ar : Option[A] = None
          var br : Option[B] = None
          
          val combiner = Actor[ Either[A,B] ] ( es ) {
              case Left(a) => br match { 
                  case None => ar = Some(a)
                  case Some(b) => eval( es )( cb(f(a, b)) )
              }
              case Right(b) => ar match{ 
                  case None => br = Some(b)
                  case Some(a) => eval( es )( cb(f(a, b)) )
              }
          }
          
          p(es)( a => combiner ! Left(a) )
          p2(es)( b => combiner ! Right(b) )
      }
  }